# 명탐정 코난 매니아 판별기 확장

## 애플리케이션 기본 정보 설정

### `config.py`: 구성정보

#### `Path`

- `Path(__file__)`: 현재 실행 중인 소스 파일의 경로(스크립트 파일의 물리적 위치)를 기준 - Jupyter Notebook에서는 `__file__` 이 정의되지 않음.
- `Path.cwd()`: 프로그램 실행 시점의 현재 작업 디렉토리 (터미널 폴더 위치)

#### `BaseSettings`

- 환경 변수(`.env` 포함) 또는 인자값을 바탕으로 설정 클래스를 정의할 때 사용하는 기본 클래스
- `BaseSettings` 를 상속받아 만든 클래스는 자동으로 환경 변수/인자에서 값을 읽어와 필드를 채워줌.

In [1]:
from pathlib import Path
from dotenv import load_dotenv
from pydantic import Field, computed_field
from pydantic_settings import BaseSettings

# .env 로드
load_dotenv()

# __file__ 이 없는 경우 (예: Jupyter) 대비 fallback
try:
    DEFAULT_BASE_DIR = Path(__file__).resolve().parents[1]
except NameError:
    DEFAULT_BASE_DIR = Path.cwd()


class Settings(BaseSettings):
    # 기본 경로
    BASE_DIR: Path = Field(default_factory=lambda: DEFAULT_BASE_DIR)

    # 모델/동작
    OPENAI_MODEL: str = "gpt-4o"
    TEMPERATURE: float = 0.5
    QUIZ_COUNT: int = 3
    QUIZ_COMMANDS: tuple[str, ...] = ("퀴즈", "퀴즈 시작")

    @computed_field
    @property
    def DATA_DIR(self) -> Path:
        path = self.BASE_DIR / "data"
        path.mkdir(parents=True, exist_ok=True)
        return path

    @computed_field
    @property
    def QUIZ_FILE(self) -> Path:
        return self.DATA_DIR / "quizzes.json"

    @computed_field
    @property
    def APPLICANT_FILE(self) -> Path:
        return self.DATA_DIR / "applicants.json"

    @computed_field
    @property
    def DB_FILE(self) -> Path:
        return self.DATA_DIR / "quiz_results.db"


# Settings 객체 생성
settings = Settings()

# 결과 출력
print(f"BASE_DIR: {settings.BASE_DIR}")
print(f"DATA_DIR: {settings.DATA_DIR}")
print(f"QUIZ_FILE: {settings.QUIZ_FILE}")
print(f"APPLICANT_FILE: {settings.APPLICANT_FILE}")
print(f"DB_FILE: {settings.DB_FILE}")

BASE_DIR: /home/lsmin/workspace/ai-agent/skala-agent-ext/src
DATA_DIR: /home/lsmin/workspace/ai-agent/skala-agent-ext/src/data
QUIZ_FILE: /home/lsmin/workspace/ai-agent/skala-agent-ext/src/data/quizzes.json
APPLICANT_FILE: /home/lsmin/workspace/ai-agent/skala-agent-ext/src/data/applicants.json
DB_FILE: /home/lsmin/workspace/ai-agent/skala-agent-ext/src/data/quiz_results.db


### `models.py`: 데이터 모델 정의

In [ ]:
from typing import Literal, Optional
from pydantic import BaseModel, Field

class RoleRoute(BaseModel):
    """역할 기반 접근 제어를 위한 경로 모델입니다."""
    role: Literal["student", "professor", "unknown"]

class ApplicantInfo(BaseModel):
    """지원자 정보를 담는 클래스입니다."""
    student_class: str = Field(description="지원자의 학급")
    student_name: str = Field(description="지원자의 이름")
    student_id: str = Field(description="지원자의 학번")
    student_phone: str = Field(description="지원자의 전화번호")

class GradingResult(BaseModel):
    """단일 문제에 대한 채점 결과를 상세히 담는 클래스입니다."""
    question_id: int = Field(description="문제의 고유 ID")
    question: str = Field(description="채점 대상 문제")
    correct_answer: str = Field(description="문제의 정답")
    user_answer: str = Field(description="사용자가 제출한 답변")
    is_correct: bool = Field(description="정답 여부")
    explanation: str = Field(description="정답에 대한 친절한 해설")

class FinalReport(BaseModel):
    """퀴즈의 모든 채점 결과와 최종 점수를 종합한 최종 보고서 클래스입니다."""
    results: list[GradingResult] = Field(description="각 문제별 채점 결과 리스트")
    total_score: str = Field(description="'총점: X/Y' 형식의 최종 점수 요약")

class ReportRequest(BaseModel):
    """최종 보고서 생성을 위한 요청 모델입니다."""
    taken_date: Optional[str] = Field(None, description="YYYY-MM-DD 또는 YYYY.MM.DD")
    student_class: Optional[str] = Field(None, description="반 (예: '2반')")
    report_type: Literal["오답", "성적", "전체"] = "전체"


## 서비스 기능 구현

### `loaders.py`: 파일 데이터 로딩

In [ ]:
import json, random
from quiz_agents.config import settings

def load_quizzes(count: int) -> list[dict]:
    """퀴즈 데이터를 JSON 파일에서 불러와 무작위로 `count`개를 선택합니다."""
    with open(settings.QUIZ_FILE, "r", encoding="utf-8") as f:
        all_q = json.load(f)
    return random.sample(all_q, min(count, len(all_q)))

def load_applicants() -> list[dict]:
    """지원자 데이터를 JSON 파일에서 불러옵니다."""    
    with open(settings.APPLICANT_FILE, "r", encoding="utf-8") as f:
        return json.load(f)


### `db.py`: DB 조작

**DB-API 2.0 (PEP 249) 호환 라이브러리** 는 두 가지 스타일을 지원

- `with closing(sqlite3.connect(settings.DB_FILE)) as conn` 패턴: 블록을 벗어나면 연결이 자동으로 닫힘

    ```python
    from contextlib import closing

    with closing(sqlite3.connect(settings.DB_FILE)) as conn:
        conn.execute("INSERT INTO quiz_results (student_name) VALUES (?)", ("홍길동",))
        conn.commit()

    ```

- `connection + cursor` 명시 선언 패턴: 여러 개의 cursor 동시 사용 가능, 개별적으로 close() 처리 필요

    ```python
    conn = sqlite3.connect(settings.DB_FILE)
    cursor = conn.cursor()
    cursor.execute("INSERT INTO quiz_results (student_name) VALUES (?)", ("홍길동",))
    conn.commit()
    cursor.close()
    conn.close()
    ```

In [ ]:
import sqlite3
import json
from contextlib import closing
from datetime import datetime

from quiz_agents.config import settings
from quiz_agents.models import FinalReport, ApplicantInfo, ReportRequest


def ensure_db():
    """데이터베이스 및 테이블 생성"""
    with closing(sqlite3.connect(settings.DB_FILE)) as conn:
        conn.execute("""
            CREATE TABLE IF NOT EXISTS quiz_results (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                taken_at TEXT NOT NULL,
                student_class TEXT,
                student_name TEXT,
                student_id TEXT,
                student_phone TEXT,
                total_score INTEGER,
                total_count INTEGER,
                details_json TEXT
            )
        """)
        conn.commit()


def save_report(applicant: ApplicantInfo, final_report: FinalReport):
    """퀴즈 결과 저장"""
    correct = sum(1 for r in final_report.results if r.is_correct)
    total = len(final_report.results)
    details = [r.model_dump() for r in final_report.results]

    with closing(sqlite3.connect(settings.DB_FILE)) as conn:
        conn.execute("""
            INSERT INTO quiz_results (
                taken_at, student_class, student_name, student_id, student_phone,
                total_score, total_count, details_json
            ) VALUES (?,?,?,?,?,?,?,?)
        """, (
            datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
            applicant.student_class,
            applicant.student_name,
            applicant.student_id,
            applicant.student_phone,
            correct, total, json.dumps(details, ensure_ascii=False)
        ))
        conn.commit()

    return correct, total


def fetch_quiz_results(report_request: ReportRequest):
    """퀴즈 결과 조회"""
    taken_date = (report_request.taken_date or "").replace("/", "-").replace(".", "-")
    student_class = report_request.student_class or ""

    sql = """
        SELECT
            student_name, student_id, student_class,
            total_score, total_count, details_json, taken_at
        FROM quiz_results
        WHERE 1=1
    """
    params: list[str] = []

    if taken_date:
        sql += " AND taken_at LIKE ?"
        params.append(f"{taken_date}%")
    if student_class:
        sql += " AND student_class = ?"
        params.append(student_class)

    sql += " ORDER BY total_score DESC, taken_at ASC"

    with closing(sqlite3.connect(settings.DB_FILE)) as conn:
        rows = conn.execute(sql, params).fetchall()

    return rows


def fetch_quiz_applicant_taken(applicant: ApplicantInfo) -> dict | None:
    """응시자의 최근 퀴즈 결과 1건 조회 (있으면 taken_at/total_score 반환)"""
    query = """
        SELECT taken_at, total_score
        FROM quiz_results
        WHERE student_id = ?
        ORDER BY id DESC
        LIMIT 1
    """
    with closing(sqlite3.connect(settings.DB_FILE)) as conn:
        row = conn.execute(query, (applicant.student_id,)).fetchone()

    if row:
        taken_at, total_score = row
        return {"taken_at": taken_at, "total_score": total_score}
    return None


# 최초 실행 시 테이블 보장
ensure_db()


### `llm.py`: LLM을 통해 JSON 형식의 응답을 받기 위한 설정 

In [ ]:
from langchain_openai import ChatOpenAI
from quiz_agents.config import settings
from quiz_agents.models import RoleRoute, ApplicantInfo, FinalReport

llm = ChatOpenAI(model=settings.OPENAI_MODEL, temperature=settings.TEMPERATURE)
llm_with_role_route = llm.with_structured_output(RoleRoute)
llm_with_applicant_info = llm.with_structured_output(ApplicantInfo)
llm_with_final_report = llm.with_structured_output(FinalReport)


## LangGraph 노드 함수 및 그래프 구성

### `state.py`: 상태 클래스 정의

In [ ]:
from typing import Literal, TypedDict, Annotated
from quiz_agents.models import ApplicantInfo, FinalReport, ReportRequest

def reduce_list(left: list, right: list) -> list:
    """두 리스트를 합칩니다."""
    return left + right

# 애플리케이션 상태 모델 정의
# 모든 필드를 선택적(total=False)으로 관리 => 상태 관리에 적합한 방식
class AppState(TypedDict, total=False):
    """
    애플리케이션의 전체 상태를 관리하는 중앙 저장소.
    Annotated를 사용하여 각 필드에 대한 설명을 타입 힌트에 포함합니다.
    """
    
    # --- 공통 및 초기 필드 ---
    user_input: Annotated[str, "사용자의 현재 입력값"]
    chat_history: Annotated[list[tuple[str, str]], "UI용 대화 기록 리스트", reduce_list]
    role: Annotated[Literal["student", "professor", "unknown"], "현재 사용자의 역할"]

    # --- 응시자(student) 흐름 관련 필드 ---
    applicant: Annotated[ApplicantInfo, "응시자 정보"]
    questions: Annotated[list[dict], "생성된 퀴즈 질문 목록"]
    quiz_index: Annotated[int, "현재 진행 중인 퀴즈의 인덱스"]
    user_answers: Annotated[list[str], "사용자가 제출한 답변 목록"]
    grading_prompt: Annotated[str, "채점을 위해 LLM에 전달할 프롬프트"]
    final_report: Annotated[FinalReport, "최종 채점 결과 보고서"]

    # --- 교수(professor) 리포트 흐름 관련 필드 ---
    report_request: Annotated[ReportRequest, "교수가 요청한 리포트 상세 정보"]

def init_state() -> dict:
    """초기 상태를 설정합니다."""
    return {
        "app_state": {
            "chat_history": [],
            "role": "unknown",
            "questions": [],
            "quiz_index": 0,
            "user_answers": [],
        }
    }


### `applicant.py`: 응시자 검증 노드 함수

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from quiz_agents.models import ApplicantInfo
from quiz_agents.nodes.state import AppState
from quiz_agents.services.loaders import load_applicants
from quiz_agents.services.llm import llm_with_applicant_info
from quiz_agents.services.db import fetch_quiz_applicant_taken


def parse_applicant_info(text: str) -> ApplicantInfo | None:
    """사용자 입력에서 지원자 정보를 추출"""

    system_message = """  
    아래 문장에서 반(student_class), 이름(student_name), 학번(student_id), 전화번호(student_phone)을 추출하세요.
    - 반: 숫자와 '반'이 포함된 문자열 (예: '1반', '2반') 
    - 이름: 한글로 된 이름
    - 학번: 'S'로 시작하는 영문자와 숫자의 조합    
    - 전화번호: 하이픈(-)이 포함될 수 있는 8개 이상의 숫자 형식

    ## 예시:
    - 입력: "1반 홍길동 S25B001 010-1111-2222"
    - 출력: JSON {{"student_class": "1반", "student_name": "홍길동", "student_id": "S25B001", "student_phone": "010-1111-2222"}}
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message.strip()),
        ("human", "{input_text}")
    ])

    try:
        response = (prompt | llm_with_applicant_info).invoke({"input_text": text})
        if not response.student_name or not response.student_id:
            return None
        return response
    except Exception:
        return None

def applicant_validator(state: AppState) -> AppState:
    """응시자 정보 검증 & 중복 응시 확인"""
    user_input = state.get("user_input", "")
    applicant = parse_applicant_info(user_input)
    if not applicant:
        return {"chat_history": [("assistant", "응시자 정보를 인식하지 못했습니다. 예) 1반 홍길동 S25B101 010-1111-1001")]}

    try:
        roster = load_applicants()
    except Exception:
        roster = []

    exists = next((r for r in roster if r.get("student_id") == applicant.student_id), None)
    if not exists:
        return {"chat_history": [("assistant", f"등록된 응시자가 없습니다: {applicant.student_id}")]}

    row = fetch_quiz_applicant_taken(applicant)
    if row:
        return {"chat_history": [("assistant", f"이미 응시 기록이 있습니다. 응시일자: {row['taken_at']}, 점수: {row['total_score']}")]}

    return {
        "applicant": applicant,
        "chat_history": [("assistant", f"{applicant.student_class} {applicant.student_name}님, '퀴즈 시작'이라고 입력하세요.")]
    }


### `quiz.py`: 퀴즈 출제 노드 함수

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from quiz_agents.models import FinalReport
from quiz_agents.nodes.state import AppState
from quiz_agents.services.loaders import load_quizzes
from quiz_agents.services.llm import llm_with_final_report


def quiz_setter(state: AppState) -> AppState:
    """퀴즈 문항 설정"""
    questions = load_quizzes()
    if not questions:
        return {
            "chat_history": [(
                "assistant",
                "퀴즈를 불러오는 데 실패했거나 풀 수 있는 문제가 없습니다."
            )],
            "questions": [],
        }
    return {
        "questions": questions,
        "quiz_index": 0,
        "user_answers": [],
        "final_report": None,
        "chat_history": [(
            "assistant",
            f"퀴즈를 시작합니다. 총 {len(questions)}문항입니다."
        )],
    }

def continue_quiz_condition(state: AppState) -> str:
    """퀴즈 계속/채점 분기 키 일치"""
    questions = state.get("questions", [])
    quiz_index = state.get("quiz_index", 0)
    if quiz_index < len(questions):
        return "continue_quiz"
    else:
        return "grade_quiz"

def quiz_popper(state: AppState) -> AppState:
    """현재 문제 출력"""
    quiz_index = state["quiz_index"]
    quiz = state["questions"][quiz_index]
    text = f"문제 {quiz_index + 1}: {quiz['question']}"
    if quiz["type"] == "multiple_choice":
        choices = [f"{i + 1}. {c}" for i, c in enumerate(quiz["choices"])]
        text += "\n" + "\n".join(choices)
    return {"chat_history": [("assistant", text)]}

def answer_collector(state: AppState) -> AppState:
    """답변 수집 및 다음 인덱스"""
    quiz_index = state.get("quiz_index", 0)
    quiz = state.get("questions", [])[quiz_index]
    user_input = state.get("user_input", "").strip()
    user_answers = state.get("user_answers", [])

    if not user_input:
        return {"chat_history": [("assistant", "답변을 입력해 주세요.")]}
    processed_answer = user_input
    if quiz["type"] == "multiple_choice":
        try:
            sel = int(user_input) - 1
            if 0 <= sel < len(quiz["choices"]):
                processed_answer = quiz["choices"][sel]
        except (ValueError, IndexError):
            pass

    user_answers.append(processed_answer)
    return {"user_answers": user_answers, "quiz_index": quiz_index + 1}

def grading_prompter(state: AppState) -> AppState:
    """채점 프롬프트 생성"""
    questions = state.get("questions", [])
    user_answers = state.get("user_answers", [])

    prompt_buff = ["지금부터 아래의 문제와 정답, 그리고 사용자의 답변을 보고 채점을 시작해주세요."]
    for i, (q, a) in enumerate(zip(questions, user_answers)):
        prompt_buff.append(f"\n--- 문제 {i + 1} ---")
        prompt_buff.append(f"문제: {q['question']}")
        if q["type"] == "multiple_choice":
            prompt_buff.append(f"선택지: {', '.join(q['choices'])}")
        prompt_buff.append(f"정답: {q['answer']}")
        prompt_buff.append(f"사용자 답변: {a}")

    return {
        "chat_history": [("assistant", "채점을 진행합니다...")],
        "grading_prompt": "\n".join(prompt_buff),
    }

def grade_reporter(state: AppState) -> AppState:
    """LLM 채점 → FinalReport 파싱"""
    system_message = """
    당신은 '명탐정 코난' 퀴즈의 전문 채점관입니다. 주어진 문제, 정답, 사용자 답변을 바탕으로 채점해주세요. 
    각 문제에 대해 정답 여부를 판단하고 친절한 해설을 덧붙여주세요. 
    모든 채점이 끝나면, 마지막에는 '총점: X/Y' 형식으로 최종 점수를 반드시 요약해서 보여줘야 합니다. 
    반드시 지정된 JSON 형식으로만 답변해야 합니다.
    """
    prompt = ChatPromptTemplate.from_messages(
        [("system", system_message), ("human", "{grading_data}")]
    )
    try:
        chain = prompt | llm_with_final_report
        report = chain.invoke({"grading_data": state["grading_prompt"]})
        return {"final_report": report}
    except Exception as e:
        print(f"채점 중 오류 발생: {e}")
        error_report = FinalReport(results=[], total_score="채점 오류가 발생했습니다.")
        return {"final_report": error_report}

def report_formatter(state: AppState) -> AppState:
    """FinalReport → 사람이 읽을 수 있는 텍스트"""
    final_report = state["final_report"]
    report_buff = ["채점이 완료되었습니다! 🎉\n"]
    if final_report and final_report.results:
        for i, res in enumerate(final_report.results):
            is_correct_text = "✅ 정답" if res.is_correct else "❌ 오답"
            report_buff.append(f"--- 문제 {i + 1} ---")
            report_buff.append(f"문제: {res.question}")
            report_buff.append(f"정답: {res.correct_answer}")
            report_buff.append(f"제출한 답변: {res.user_answer}")
            report_buff.append(f"결과: {is_correct_text}")
            report_buff.append(f"해설: {res.explanation}\n")
        report_buff.append(f"**{final_report.total_score}**")
    else:
        report_buff.append("채점 결과를 생성하는 데 실패했습니다.")
    report_buff.append("\n퀴즈를 다시 시작하려면 '퀴즈 시작'이라고 입력해주세요.")
    return {"chat_history": [("assistant", "\n".join(report_buff))]}



### `report.py`: 채점 결과 리포트 노드 함수

In [ ]:
import re, json
from quiz_agents.models import ReportRequest
from quiz_agents.nodes.state import AppState
from quiz_agents.services.db import save_report, fetch_quiz_results

def grade_report_saver(state: AppState) -> AppState:
    """채점 결과 DB 저장 (숫자 집계로 저장)"""
    applicant = state.get("applicant")
    final_report = state.get("final_report")
    if applicant and final_report and final_report.results:
        save_report(applicant, final_report)
        return {"chat_history": [("assistant", "채점 결과가 성공적으로 저장되었습니다.")]}
    else:
        return {"chat_history": [("assistant", "채점 결과를 저장하는 데 실패했습니다.")]}    

def report_request_parser(state: AppState) -> AppState:
    """리포트 요청 파싱"""
    user_input = state.get("user_input", "")
    date_match = re.search(r"(\d{4}[-/.]\d{2}[-/.]\d{2})", user_input)
    taken_date = date_match.group(1).replace(".", "-") if date_match else ""
    class_match = re.search(r"(\d+반)", user_input)
    student_class = class_match.group(1) if class_match else ""
    if "오답" in user_input:
        report_type = "오답"
    elif "성적" in user_input:
        report_type = "성적"
    else:
        report_type = "전체"
    report_request = ReportRequest(taken_date=taken_date, student_class=student_class, report_type=report_type)
    return {"report_request": report_request}

def _rank_table(rows: list) -> str:
    """성적 순위 테이블 생성"""
    parts = ["### 성적 순위 (높은 점수 우선)", "이름 | 학번 | 반 | 점수 | 일시", "---|---|---|---|---"]
    for s_name, s_id, s_class, t_score, t_count, _, taken_at in rows:
        parts.append(f"{s_name} | {s_id} | {s_class} | {t_score}/{t_count} | {taken_at}")
    return "\n".join(parts)

def _wrong_table(rows: list) -> str:
    """오답률 상위 문항 테이블 생성"""
    agg: dict[str, list[int]] = {}
    for *_, details_json, _ in rows:
        try:
            details = json.loads(details_json)
            for d in details:
                qid = f"{d.get('question_id', '?')}.{d.get('question', '')[:16]}"
                is_correct = d.get("is_correct", False)
                if qid not in agg:
                    agg[qid] = [0, 0]
                agg[qid][1] += 1
                if not is_correct:
                    agg[qid][0] += 1
        except Exception:
            continue
    items = []
    for qid, (wrong, total) in agg.items():
        rate = (wrong/total*100) if total else 0.0
        items.append({"qid": qid, "wrong": wrong, "total": total, "rate": rate})
    items.sort(key=lambda x: x["rate"], reverse=True)
    parts = ["\n### 오답률 상위 문항", "문항 | 오답수/응시수 | 오답률(%)", "---|---|---"]
    for it in items[:20]:
        parts.append(f"{it['qid']} | {it['wrong']}/{it['total']} | {it['rate']:.1f}")
    return "\n".join(parts)

def report_generater(state: AppState) -> AppState:
    """요청 조건에 맞는 리포트 생성"""
    report_request = state.get("report_request")
    if not report_request:
        return {"chat_history": [("assistant", "리포트 요청을 파싱하지 못했습니다.")]}
    quiz_results = fetch_quiz_results(report_request)
    if not quiz_results:
        return {"chat_history": [("assistant", "해당 조건의 응시 기록이 없습니다.")]}
    report_outputs = []
    report_type = report_request.report_type
    if report_type in ("성적", "전체"):
        report_outputs.append(_rank_table(quiz_results))
    if report_type in ("오답", "전체"):
        report_outputs.append(_wrong_table(quiz_results))
    final_report = "\n\n".join(report_outputs)
    return {"chat_history": [("assistant", final_report)]}


### `routing.py`: 사용자 역할에 따른 라우팅 처리 노드 함수

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from quiz_agents.config import settings
from quiz_agents.services.llm import llm_with_role_route
from quiz_agents.nodes.state import AppState

def classify_role(text: str) -> str:
    """ 사용자 입력을 분석하여 역할을 분류하는 함수입니다."""

    system_message = """  
    당신은 사용자 유형을 분류하는 매우 정확한 라우터입니다. 사용자의 입력을 보고 'student', 'professor', 'unknown' 중 하나로 분류해주세요.

    ## 분류 기준:
    1. 'student': 반, 이름, 학번 등 개인정보를 포함하여 퀴즈 응시를 시도하는 경우.
    2. 'professor': 날짜, 반, '리포트' 또는 '성적'과 같은 키워드를 포함하여 결과를 조회하려는 경우.
    3. 'unknown': 위 두 경우에 해당하지 않는 모든 애매한 경우.

    ## 예시:
    - 입력: "1반 홍길동 S25B001 010-1111-2222", 분류: 'student'
    - 입력: "2025-07-07 2반 성적 순위 리포트 좀 보여줘", 분류: 'professor'
    - 입력: "안녕하세요", 분류: 'unknown'
    - 입력: "퀴즈를 풀고 싶어요.", 분류: 'unknown'

    ## 출력 형식:
    JSON {{"role": "student|professor|unknown"}} 한 값만 주세요.
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system", system_message.strip()),
        ("human", "{input_text}")
    ])
    try:
        response = (prompt | llm_with_role_route).invoke({"input_text": text})
        return response.role
    except Exception:
        return "unknown"

def entry_router(state: AppState) -> str:
    """역할 분류 및 진입점 라우터"""

    ui = state.get("user_input", "").strip()

    # 1) 퀴즈 시작 명령 우선
    if any(cmd == ui for cmd in settings.QUIZ_COMMANDS):
        return "quiz_entry"

    # 2) 문제 진행 중이면 답변 입력으로 라우팅
    qs = state.get("questions", [])
    qi = state.get("quiz_index", 0)
    if qs and (0 <= qi < len(qs)):
        return "answer_entry"

    # 3) 역할 분류
    role = classify_role(ui) if ui else "unknown"
    if role == "student":
        return "student_entry"
    elif role == "professor":
        return "professor_entry"
    else:
        return "unknown_entry"

def entry_helper(state: AppState) -> AppState:
    """알 수 없는 역할에 대한 도움말"""
    
    help_text = (
        "학생은 `1반 홍길동 S25B101 010-1111-1001` 처럼 본인 정보를 입력하세요.\n"
        "교수는 `2025-07-07 2반 리포트 출력`처럼 날짜와 반을 포함해 입력하세요.\n"
        "퀴즈를 시작하려면 `퀴즈 시작`이라고 입력하세요."
    )
    return {"chat_history": [("assistant", help_text)]}


### `graph.py`: 노드 함수 연결 및 그래프 생성 

In [ ]:
from langgraph.graph import StateGraph, END
from quiz_agents.nodes.state import AppState
from quiz_agents.nodes.routing import entry_router, entry_helper
from quiz_agents.nodes.applicant import applicant_validator
from quiz_agents.nodes.quiz import (
    quiz_setter, quiz_popper, answer_collector,
    continue_quiz_condition, grading_prompter, grade_reporter, report_formatter
)
from quiz_agents.nodes.report import grade_report_saver, report_request_parser, report_generater

def build_graph():
    graph = StateGraph(AppState)

    # 노드 추가
    graph.add_node("entry_helper", entry_helper)
    graph.add_node("applicant_validator", applicant_validator)
    graph.add_node("quiz_setter", quiz_setter)
    graph.add_node("quiz_popper", quiz_popper)
    graph.add_node("answer_collector", answer_collector)
    graph.add_node("grading_prompter", grading_prompter)
    graph.add_node("grade_reporter", grade_reporter)
    graph.add_node("grade_report_saver", grade_report_saver)
    graph.add_node("report_formatter", report_formatter)
    graph.add_node("report_request_parser", report_request_parser)
    graph.add_node("report_generater", report_generater)

    # 조건부 엔트리
    graph.set_conditional_entry_point(
        entry_router,
        {
            "quiz_entry": "quiz_setter",
            "answer_entry": "answer_collector",
            "student_entry": "applicant_validator",
            "professor_entry": "report_request_parser",
            "unknown_entry": "entry_helper",
        },
    )

    # 엣지
    graph.add_edge("quiz_setter", "quiz_popper")
    graph.add_edge("quiz_popper", END)  # 문제 출력 후 턴 종료
    graph.add_edge("entry_helper", END)

    graph.add_conditional_edges(
        "answer_collector",
        continue_quiz_condition,
        {"continue_quiz": "quiz_popper", "grade_quiz": "grading_prompter"},
    )
    graph.add_edge("grading_prompter", "grade_reporter")
    graph.add_edge("grade_reporter", "grade_report_saver")
    graph.add_edge("grade_report_saver", "report_formatter")
    graph.add_edge("report_request_parser", "report_generater")
    graph.add_edge("report_formatter", END)
    graph.add_edge("report_generater", END)

    return graph.compile()


### 그래프 시각화

In [ ]:
from langchain_teddynote.graphs import visualize_graph

quiz_app_graph = build_graph()
visualize_graph(quiz_app_graph)

## 애플리케이션 진입 방식 구현

### `ui/app.py`: Gradio UI 모드

In [ ]:
import gradio as gr
from quiz_agents.nodes.graph import build_graph
from quiz_agents.nodes.state import init_state
from quiz_agents.config import settings

quiz_app = build_graph()

def chat_fn(user_input, state):
    """사용자 입력을 처리하고 상태를 업데이트합니다."""

    app_state = state["app_state"]
    app_state.setdefault("chat_history", []).append(("user", user_input))
    app_state["user_input"] = user_input

    new_state = quiz_app.invoke(app_state)
    state["app_state"] = new_state

    chat_display = [{"role": r, "content": c} for r, c in new_state.get("chat_history", [])]
    return chat_display, state

def build_ui():
    """Gradio UI를 빌드합니다."""
    
    with gr.Blocks(theme=gr.themes.Soft()) as demo:
        gr.Markdown("""
        ### 🧩 멀티 에이전트 퀴즈/리포트 (LangGraph)
        - 학생 예: `1반 홍길동 S25B101 010-1111-1001` → 확인 후 `퀴즈 시작`
        - 교수 예: `2025-07-07 2반 리포트 출력` / `오답 리포트` / `성적 리포트`
        """)
        chatbot = gr.Chatbot(
            label="명탐정 코난 퀴즈 챗봇",
            height=400,
            avatar_images=(str(settings.DATA_DIR/"avatar_user.png"), str(settings.DATA_DIR/"avatar_conan.png")),
            type="messages",
        )
        txt = gr.Textbox(placeholder="메시지를 입력해보세요!", show_label=False)
        state = gr.State(init_state())

        txt.submit(chat_fn, inputs=[txt, state], outputs=[chatbot, state])
        txt.submit(lambda: "", None, txt)
    return demo


### `api/app.py`: FastAPI API 모드

In [ ]:
from fastapi import FastAPI
from quiz_agents.nodes.graph import build_graph
from quiz_agents.nodes.state import init_state

app = FastAPI()
quiz_app = build_graph()

@app.get("/")
def read_root():
    return {"message": "명탐정 코난 Quiz Agents API가 동작 중입니다."}

@app.post("/chat")
def chat(user_input: str, state: dict = None):
    """퀴즈 챗봇과 상호작용하는 엔드포인트."""
    
    if state is None:
        state = init_state()

    app_state = state["app_state"]
    app_state.setdefault("chat_history", []).append(("user", user_input))
    app_state["user_input"] = user_input

    new_state = quiz_app.invoke(app_state)
    state["app_state"] = new_state

    chat_display = [{"role": r, "content": c} for r, c in new_state.get("chat_history", [])]
    return {"chat": chat_display, "state": state}


## 애플리케이션 실행

### `main.py`: 진입 모드에 따라 실행

main.py가 있는 폴더로 이동 후 애플리케이션 기동

```bash
python main.py [ui|api]
```